In [17]:
import faiss
import numpy as np
import os

In [18]:
import torch
import transformers

In [19]:
# 设置 Hugging Face 的镜像站点
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from sentence_transformers import SentenceTransformer

In [20]:
# 初始化模型
model_path = "./stsb/models"  # Hugging Face 模型名称
model = SentenceTransformer(model_path)  # 在程序开始时加载模型

In [21]:
# 读取文档
def read_documents(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()  # 直接读取整个文档内容
    return content

# 递归分块函数
def recursive_chunking(text, max_chunk_size=100, separators=None):
    if separators is None:
        separators = ["\n\n", "\n", "。", "，", " "]  # 默认分隔符
    
    chunks = []
    current_chunk = ""
    
    # 如果没有分隔符，直接返回整个文本
    if not separators:
        chunks.append(text)
        return chunks
    
    # 使用第一个分隔符进行分割
    separator = separators[0]
    parts = text.split(separator)
    
    for part in parts:
        # 如果当前块加上新部分的大小超过最大块大小，则递归分割
        if len(current_chunk) + len(part) > max_chunk_size:
            if current_chunk:
                chunks.append(current_chunk)
            chunks.extend(recursive_chunking(part, max_chunk_size, separators[1:]))
            current_chunk = ""
        else:
            current_chunk += part + separator if separator else part
    
    # 添加最后一个块
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

# 分块知识库
def chunk_knowledge_base(content, chunk_strategy="recursive", max_chunk_size=100):
    chunks = []
    if chunk_strategy == "recursive":
        chunks = recursive_chunking(content, max_chunk_size)
    return chunks

# 向量化分块
def vectorize_chunks(chunks):
    chunk_vectors = model.encode(chunks, convert_to_numpy=True).astype('float32')
    return chunk_vectors

# 创建 HNSW 索引
def create_hnsw_index(doc_vectors, M=16, efConstruction=200):
    d = doc_vectors.shape[1]  # 向量维度
    index = faiss.IndexHNSWFlat(d, M)  # 使用 HNSW 索引
    index.hnsw.efConstruction = efConstruction  # 设置构建参数
    index.add(doc_vectors)  # 添加向量到索引
    return index

# 保存索引和分块
def save_index_and_chunks(index, chunks, index_path="faiss_index.bin", chunks_path="chunks.txt"):
    faiss.write_index(index, index_path)
    with open(chunks_path, "w", encoding="utf-8") as f:
        for chunk in chunks:
            f.write(chunk + "\n")

# 加载索引和分块
def load_index_and_chunks(index_path="faiss_index.bin", chunks_path="chunks.txt"):
    index = faiss.read_index(index_path)
    with open(chunks_path, "r", encoding="utf-8") as f:
        chunks = [line.strip() for line in f if line.strip()]
    return index, chunks

# 检索相似分块
def search_related_chunks(query, index, chunks, k=3, efSearch=100, similarity_threshold=0.00):
    query_vector = model.encode([query], convert_to_numpy=True).astype('float32')  # 使用全局的 model
    
    # 设置搜索参数
    index.hnsw.efSearch = efSearch
    
    # 检索
    distances, indices = index.search(query_vector, k)  # 先检索前 k 个结果
    
    # 将距离转换为相似度（Faiss 返回的是 L2 距离，越小表示越相似）
    similarities = 1 / (1 + distances)  # 将距离转换为相似度
    
    # 筛选满足相似度阈值的结果
    filtered_results = []
    for i in range(len(indices[0])):
        if similarities[0][i] >= similarity_threshold:
            filtered_results.append((chunks[indices[0][i]], similarities[0][i]))
    
    # 如果满足阈值的结果超过 3 个，只返回前 3 个
    if len(filtered_results) > 3:
        filtered_results = filtered_results[:3]
    
    # 返回结果（包含文本和相似度）
    return filtered_results

# 主函数：构建 HNSW 数据库
def build_hnsw_database(file_path, chunk_strategy="recursive", max_chunk_size=100):
    # 读取文档
    content = read_documents(file_path)
    print("文档读取完成")
    
    # 分块知识库
    chunks = chunk_knowledge_base(content, chunk_strategy, max_chunk_size)
    print(f"知识库分块完成，共生成 {len(chunks)} 个分块")
    
    # 向量化分块
    chunk_vectors = vectorize_chunks(chunks)
    print("分块向量化完成")
    
    # 创建并保存 HNSW 索引
    index = create_hnsw_index(chunk_vectors)
    save_index_and_chunks(index, chunks)
    print("HNSW 数据库已创建并保存")
    return index, chunks

# 主函数：加载 HNSW 数据库并检索相关文档
def search_in_hnsw_database(query, k=2):
    # 加载 HNSW 索引
    index, chunks = load_index_and_chunks()
    # 检索相关文档
    related_chunks = search_related_chunks(query, index, chunks, k)
    return related_chunks


In [22]:
import requests
import json

In [23]:
if __name__ == "__main__":
    # 构建 HNSW 数据库
    index, chunks = build_hnsw_database("./RAGBase.txt", chunk_strategy="recursive", max_chunk_size=100)
    
    # 进入查询循环
    while True:
        # 读取用户输入
        query = input("请输入查询内容（输入 'exit' 退出）：")
        
        # 如果输入 'exit'，退出程序
        if query == "exit":
            print("程序已退出。")
            break


        prompt = "你是中方的谈判专家，当前中印对话内容如下：" + query + "。请你进行总结，请你分析出当前对方的心理状态以及主要矛盾点，根据当前背景给出你的分析结论，用一段连续的话说明。"
        
        url = "http://localhost:11535/api/generate"
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "model": "qwen:14b",
            "prompt": prompt,
            "temperature": 0.8
        }

        try:
            # 发送 POST 请求
            response = requests.post(url, headers=headers, json=data)
            # 检查请求是否成功，如果不成功会抛出异常
            response.raise_for_status()

            # 初始化一个空字符串，用于存储最终拼接的结果
            combined_response = ""
            # 将响应内容按行分割，得到一个包含每行内容的列表
            lines = response.text.splitlines()
            for line in lines:
                try:
                    # 尝试将每行内容解析为 JSON 对象
                    json_obj = json.loads(line)
                    # 从解析后的 JSON 对象中提取 response 字段的值
                    combined_response += json_obj["response"]
                except json.JSONDecodeError:
                    # 如果解析失败，打印提示信息
                    print(f"无法解析行: {line}")

            # 打印拼接后的完整内容
            print(combined_response)

        except requests.exceptions.HTTPError as http_err:
            # 处理 HTTP 请求错误
            print(f"HTTP 错误发生: {http_err}")
        except Exception as err:
            # 处理其他异常
            print(f"发生其他错误: {err}")

        # 检索相关文档
        related_docs = search_related_chunks(combined_response, index, chunks, k=3)
        related_doc_texts = [doc[0] for doc in related_docs]
        print("与查询最相关的文档原文：")
        for doc, similarity in related_docs:
            print(f"相似度: {similarity:.4f}\n文档: {doc}")
        print("-" * 50)  # 分隔线

        prompt2 = "你现在作为中方，当前问题为：" + query + "，与该问题相关的文档内容如下：" + " ".join(related_doc_texts) + "。请你以第一人称的方式，直接给出回答。"
        url = "http://localhost:11535/api/generate"
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "model": "qwen:14b",
            "prompt": prompt2,
            "temperature": 0.8
        }

        try:
            # 发送 POST 请求
            response = requests.post(url, headers=headers, json=data)
            # 检查请求是否成功，如果不成功会抛出异常
            response.raise_for_status()

            # 初始化一个空字符串，用于存储最终拼接的结果
            combined_response = ""
            # 将响应内容按行分割，得到一个包含每行内容的列表
            lines = response.text.splitlines()
            for line in lines:
                try:
                    # 尝试将每行内容解析为 JSON 对象
                    json_obj = json.loads(line)
                    # 从解析后的 JSON 对象中提取 response 字段的值
                    combined_response += json_obj["response"]
                except json.JSONDecodeError:
                    # 如果解析失败，打印提示信息
                    print(f"无法解析行: {line}")

            # 打印拼接后的完整内容
            print(combined_response)

        except requests.exceptions.HTTPError as http_err:
            # 处理 HTTP 请求错误
            print(f"HTTP 错误发生: {http_err}")
        except Exception as err:
            # 处理其他异常
            print(f"发生其他错误: {err}")

    else:
        print("已退出程序")

文档读取完成
知识库分块完成，共生成 1230 个分块
分块向量化完成
HNSW 数据库已创建并保存
在中印谈判的对话中，可以看出双方立场坚定且敏感。印度代表试图以宗教文化为借口，重新界定边界，表现出对传统信仰的强调和对领土诉求的坚持。

而中方代表则坚决维护历史条约和国际法，明确拒绝了随意更改边界的提议，表明了中国政府在领土主权问题上的不容商量的态度。

总结起来，当前印方的心理状态是试图通过情感诉求争取优势，而主要矛盾点在于宗教与领土之间的界限。而中方的立场则是坚定且有原则性的，不容任何侵犯主权的行为。
与查询最相关的文档原文：
相似度: 0.0151
文档: 本协定须得到双方批准，自互换批准书之日起生效。
本协定持续有效，直至协定的任何一方决定中止本协定并提前六个月书面通知另一方，则本协定在通知六个月后失效。
本协定经双方书面同意后，可进行修改和补充。

相似度: 0.0135
文档: 应方面，在当时基本上依靠印度的转运和进口。 尼赫鲁充分意识到西藏在粮食和物质供应上对  印度的依赖，同时也充分意识到中国在这些问  题上对印度的依赖。
相似度: 0.0130
文档: 神，对本国的划界主张作出富有意义的和双方均能接受的调整”。因此，如何对中印传 统 习 惯 线 进 行调整，也是我国在中印边界谈判中要着重解决的法律问题。
--------------------------------------------------
作为中方代表，在面对印方关于宗教文化影响锡金段边界主张时，我坚定地回应：

我们的立场是明确的。历史和国际法已经确定了锡金段边界的归属，这不应受到任何宗教信仰的影响。

我们尊重所有国家的文化多样性，但这种尊重不能成为改变既有领土疆界的借口。印方应停止无端的争议制造，而是寻求通过对话与合作解决分歧。
程序已退出。
